# Programming Paradigms

In this lecture, we will look at programming pardigms that can help us write and modify embedded libraries and languages. The goal of this lecture is not to go over an overview of programming paradigms in general, but only the ones that are relevant to embeddings. The notes assumes some familiarity with the most popular programming paradigms (imperative, functional, and object-oriented), but will provide resources about these paradigms when they are mentioned.

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<h2> Motivation </h2>

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

Programming paradigms allow us to categorize programming languages into hirearchical families. The hirearchical aspect of this categorization is important since the categorization is relative. It is very similar to the <a href="https://en.wikipedia.org/wiki/Taxonomy_(biology)#Kingdoms_and_domains">taxonomy of biology</a>, in which al living beings are classified into categories: when compared to another kingdom, the members of a single kingdom seem relatively similar. However, if the members are compared among themselves, relative differences become important, and can be used to further classify the members into Phylumns, Classes, etc.

The goals of any such classification is to simplify comparisions of members. There are several hundreads programming languages around. It is unrealistic to expect developers to learn about all (or even a majority) of them. However, it is still pretty useful for developers to understand the general tradeoffs between these programming languages, so that they can make informed decisions in what languages to use, or what resources to look for.

Programming Paradigms provide the developers with exactly that. Members of a programming paradigms have certain similarties, that allow them to be abstracted together into a paradigm. The paradigm exhibits the same general tradeoffs when compared to other paradigms. The more detailed we make our paradigm, the few languages it contains, and the more accurate its description of tradeoffs is. However, making paradigms too detailed will entail having many paradigms, making it unrealistic for developers to understand all of them. Hence, paradigms are structured hierarchicaly, where the top level paradigms are abstract and few, and the lower level ones are more detailed and plenty.

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

<h2> Taxonomy </h2>

</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">

This <a href="https://en.wikipedia.org/wiki/Programming_paradigm">Wikipedia article</a> shows a snippet of the programming paradigms hierarchy. Some of these paradigms are less popular than others, while some are very standard, and you can expect to see them in any introductory course on programming languages, these include (among others): imperative, functional, and object-orient programming.

With the advances made in the design of programming languages, the more recent programming languages are becoming increasingly multi-paradigms: they include elements from different paradigms which can be used in tandem. The main advantage of this is increased flexibility and friendliness. Developers using a programming language can choose to use features more applicable to their particular task, or to mix different features for increased productivity and simplicity.

An example of such a multi-paradigm language is Python: it supports typical structured imperative programming, with basic conditional and loop constructs, it also supports classes and complex inheritance (object oriented-programming), while also supporting the use of functions as "first-class citizens", where functions can be assigned to variables and passed around or manipulated, including high level combinators (such as map and reduce), which are concepts from functional programming.

Another example is Javascript, which supports similar features to Python, but relies on prototype based inheritance, although newer Javascript standards support classes as well.

We assume some familiarity with these three paradigms, and will refocus our attention to a few less known paradigms that help us develop embedded languages. If you need a refresher on these paradigms, please look at these wikipedia articles: <a href="https://en.wikipedia.org/wiki/Imperative_programming">Imperative</a>, <a href="https://en.wikipedia.org/wiki/Functional_programming">Functional</a>, and <a href="https://en.wikipedia.org/wiki/Object-oriented_programming">Object-oriented</a> programming.

</div>

# Scenario

Our running example is the following: assume we are a software engineer in some company, our code base relies on some sophisticated library, but it is exhibiting some weird slow-downs. Code that we believe to be efficient is running very slowly. We suspect that we are using some of these library functions wrong, which is causing that slow down, and we are trying to determine which of these calls is causing this slow down.

Our first instinct is to surround every call to a library function with some timing statements, and print the time it takes for this function call to return to the screen. This is not the best solution, since it involves modifying our code. In fact, if our code is big, and makes frequent calls to this library, this will entail modifying a huge portion of our code. This is time consuming.

Another instinct is to modify the library, so that every function in it does that timing and prints the time to the screen. This is not great either. If the library is large, we will have to spend a lot of time adding our modifications. Furthermore, the library may be closed source and thus unmodifiable easily. Finally, if our task was more complicated than just timing (perhaps some complex analysis), we run the risk of introducing bugs into the library.

Instead, we will rely on various programming paradigms features to assist us in our task. An **ideal** solution will exhibit these properties:

1. **Compact:** its size is independent of both the size of the library and our code.
2. **Library-agnostic:** It does not entail modifying the library and is compatible with closed source libraries, and can be re-used for different libraries with minimal effort.
3. **Friendly:** It minimally modifies our company's code, and can be re-used for other code bases with minimal changes.

### Example Library

Below is an example library that we would like to time. The library consists of a single class with several functions, some of them calling each other.

In [1]:
class MyClass(object):
    def __init__(self, value):
        self.value = value

    def function1(self):
        return [self.value] * 1000000000

    def function2(self):
        return [1] * self.value

    def function3(self):
        return [0] + self.function1()[:100000] + self.function2()

### Example client code

The following is an example client code. Running this code may take some time.

In [2]:
def original_code():
    obj1 = MyClass(10)
    obj2 = MyClass(5)

    print(obj1.function1()[0], '\n')
    print(obj1.function2()[0], '\n')
    print(obj1.function3()[0], '\n')

    print(obj2.function1()[0], '\n')
    print(obj2.function2()[0], '\n')
    print(obj2.function3()[0], '\n')

original_code()

10 

1 

0 

5 

1 

0 



# Solution 1: Inheritance

This first solution tries to mimic a simple intuition: we would like to put time and print statements inside every function in our library. Since we cannot modify the library directly, we can resort to inherit its classes inside our code, and write our modifications in it. Below is one way to acheive this. Note that this relies on concepts from the **Object-oriented programming paradigm**.

In [3]:
import time

class MyClass2(MyClass):
    def __init__(self, value):
        super().__init__(value)

    def function1(self):
        t = time.time()
        ret = super().function1()
        print('function1', int(time.time() - t))
        return ret

    def function2(self):
        t = time.time()
        ret = super().function2()
        print('function2', int(time.time() - t))
        return ret

    def function3(self):
        t = time.time()
        ret = super().function3()
        print('function3', int(time.time() - t))
        return ret

We must make sure our code uses the new sub class we created, we will have to modify our code to ensure this. In reality, if the code is structured into modules, this modification can be small. Since we can just modify the import statements at the top of every file to import our class instead of that from the library. Running this code will take some time.

In [4]:
def modified_code():
    obj1 = MyClass2(10)
    obj2 = MyClass2(5)

    print(obj1.function1()[0], '\n')
    print(obj1.function2()[0], '\n')
    print(obj1.function3()[0], '\n')

    print(obj2.function1()[0], '\n')
    print(obj2.function2()[0], '\n')
    print(obj2.function3()[0], '\n')

modified_code()

function1 2
10 

function2 0
1 

function1 2
function2 0
function3 4
0 

function1 2
5 

function2 0
1 

function1 2
function2 0
function3 4
0 



## Evaluation

Our first solution is not ideal. It does not have compactness: we have to extend every class in our library and override every one of it. It also entails some amount of modification to our client code. However, it is relatively easy to understand and implement, and thus is not very bug-prone. Furthermore, it shows the time for internal library calls: function3 in our library calls both functions 1 and 2 inside it. When our client code calls function3, that call in addition to the two calls inside are all timed and printed to the screen.

# Solution 2: Reflection (Meta-programming)

Meta-programming is a programming paradigm, where one can write regular program code (programs that manipulate user data), accompanied by meta code, which manipulate and transform the regular programs. You can have a hirearchy of such levels in one program.

Meta-programming has some support in modern programming language: one of the most popular ways this is supported is via **[reflection](https://www.geeksforgeeks.org/reflection-in-python/)**, which allows developers to use the language's constructs (such as loops or if statements) to dynamically read and write into properties of existing code. This is supported in Python, JavaScript, Java, and other languages to various degrees.

Our idea for the reflection solution is simple. In our inheritance-based solution, most of our added code is repetitive. We would like to write that code once as a single function that we can re-use later for all library functions.

In [5]:
def wrap(func_name, func):
    # the repetitive function from our inheritance solution
    # parameterized by the name and actual function of the function we are going
    # to be modifying in our library
    def repetitive(*args, **kwargs):
        # using *args and **kwargs in this popular pattern allows us to support
        # functions that take any number of positional arguments as well as keyword arguments
        # (i.e. arguments defined using the 'arg=default' style).
        t = time.time()
        r = func(*args, **kwargs) # pass whatever arguments we received to func
        print(func_name, int(time.time() - t))
        return r

    return repetitive

Now, we can re-use the above repetitive function in our inheritance solution, to make our implementation a bit smaller. However, we can do something better. We can use reflection to loop over all functions of our library class, and change them to use the reptitive function.

In [6]:
library_classes = [ MyClass ] # we can support many library classes easily
for library_class in library_classes:
    print(dir(library_class))
    # loop over all attributes in a class, including
    # internal functions and non-function attributes
    for attr_name in dir(library_class):
        attr = getattr(library_class, attr_name)
        # make sure the attribute represents a public function
        if attr_name.startswith('__') or not callable(attr):
            continue

        # change the attribute function to be our repetitive function!
        setattr(library_class, attr_name, wrap(attr_name, attr))

original_code()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'function1', 'function2', 'function3']
function1 2
10 

function2 0
1 

function1 2
function2 0
function3 4
0 

function1 2
5 

function2 0
1 

function1 2
function2 0
function3 4
0 



## Evaluation

This solution is far more ideal than the previous one.

1. It is **compact**: the same code we wrote can support libraries of any number of classes and functions.
2. It is **library-agnostic**: if we update the library version or switch to another library, the same code will work, since it does not refer to any library specific function or feature by name.
3. It is **friendly**: it does not require us to modify our client code, we only need to add some preamble at the start that performs the reflection. Note that the code snippet above still calls 'original_call'.

However, our solution is highly reflexive, which means it is more abstract. It is a bit harder to understand and write for developers who have not used reflection before. For more complicated tasks, our reflection may be a bit more complicated, making it somewhat bug prone.

Below we re-define our library to remove the effects of reflection into the coming solutions.

In [7]:
class MyClass(object):
    def __init__(self, value):
        self.value = value
    def function1(self):
        return [self.value] * 1000000000
    def function2(self):
        return [1] * self.value
    def function3(self):
        return [0] + self.function1()[:100000] + self.function2()

# Solution 3: Instrumentation

[Instrumentation](https://en.wikipedia.org/wiki/Instrumentation_(computer_programming)) is not a programming paradigm. However, it is a very popular technique that can be used to modify a language to support new paradigms, or to customize its behavior for testing, monitoring, and tracing. It is a very powerful technique. It has a static analysis component very similar to what we saw in the previous lecture.

Instrumentation works via writing some code to parse and manipulate client or library code, and then running the new manipulated code. In our case, we will parse the client code, and transform it so that every call to a library function is surrounded by a print statement automatically. This automates our first instinct for a solution.

Note that languages have variable levels of support for instrumentation. Some languages allows you to write the instrumentation code as part of the client code, and run it during the program run time, this can be acheived in Python and JavaScript (via [Babel](https://babeljs.io/)). Other languages may allow you to write instrumentation code in separate modules, that are run automatically during a compilation stage prior to running the application, this can be done in modern Java using [annotation processing](https://medium.com/@jintin/annotation-processing-in-java-3621cb05343a) (e.g. [Project Lombok](https://projectlombok.org/)). Other languages do not support instrumentation directly. Instead, they provide external tools that can be used to perform instrumentation when desired prior to or after compilation, examples of this include using [Apache BCEL](https://commons.apache.org/proper/commons-bcel/) for manipulating Java Byte code after compilation, or [LLVM-level instrumentation](https://stackoverflow.com/questions/7806689/instrumenting-c-c-code-using-llvm) of machine code.

**Disclaimer:** _The implementation details may be interesting to you if you would like to build your instrumentation code for a project. However, it is more important that you understand the concept of instrumentation and its power. Feel free to skim through the instrumentation actual code as you see fit._

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
    
<h2> Instrumentation Implementation </h2>

Our example code below demonstrates how instrumentation can be used for our task. Our entry point function parses the source code of our client function, and loops over every function definition in that code. It instruments the body of each function, then dumps the result back to the output code.

</div>

In [8]:
import inspect, ast, astor

def instrument(code):
    code = inspect.getsource(code)
    tree = ast.parse(code)

    print('input code ----------------------------')
    print(code)
    print('---------------------------------------')

    # find function definitions and instrument the body of each function
    statements = []
    for statement in tree.body:
        if isinstance(statement, ast.FunctionDef):
            print('hello')
            statement = instrument_body(statement)
        statements.append(statement)

    tree.body = statements
    code = astor.to_source(tree)

    print('output code --------------------------')
    print(code)
    print('---------------------------------------')

    return code

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">

Every function body is instrumented as follows: we loop over every statement in that function. We determine if that statement may require instrumentation, by checking if it is an expression that contains a function call. These statements are then instrumented independently.

</div>

In [9]:
def instrument_body(function_def):
    statements = []
    for statement in function_def.body:
        if isinstance(statement, ast.Expr) and isinstance(statement.value, ast.Call):
            statements += instrument_statement(statement, statement.value)
        else:
            # does not require an instrumentation because it is not a function call
            statements.append(statement)

    function_def.body = statements
    return function_def

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">

Every statement that potentially needs instrumentation is further analyzed. We check that this statement does indeed call a method of some class by invoking its name. If it does, then we surround the statement with a time and print statement.

</div>

In [10]:
def instrument_statement(statement, statement_value):
    # double check that the statement really calls a method
    for node in ast.walk(statement_value.args[0]):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
            # find the method name
            name = astor.to_source(node.func).strip()
            # create a time and print statement
            time_statement = ast.parse('__t = time.time()')
            print_statement = ast.parse('print("'+name+'", int(time.time() - __t))')
            # surround original statement by time and print!
            return [time_statement, statement, print_statement]

    return [statement]

## Evaluation and Usage

Finally, we can call our instrumentation code from our client code before running it as follows:

In [11]:
copy = original_code
instrumented_code = instrument(original_code)
exec(instrumented_code)

original_code()

input code ----------------------------
def original_code():
    obj1 = MyClass(10)
    obj2 = MyClass(5)

    print(obj1.function1()[0], '\n')
    print(obj1.function2()[0], '\n')
    print(obj1.function3()[0], '\n')

    print(obj2.function1()[0], '\n')
    print(obj2.function2()[0], '\n')
    print(obj2.function3()[0], '\n')

---------------------------------------
hello
output code --------------------------
def original_code():
    obj1 = MyClass(10)
    obj2 = MyClass(5)
    __t = time.time()
    print(obj1.function1()[0], '\n')
    print('obj1.function1', int(time.time() - __t))
    __t = time.time()
    print(obj1.function2()[0], '\n')
    print('obj1.function2', int(time.time() - __t))
    __t = time.time()
    print(obj1.function3()[0], '\n')
    print('obj1.function3', int(time.time() - __t))
    __t = time.time()
    print(obj2.function1()[0], '\n')
    print('obj2.function1', int(time.time() - __t))
    __t = time.time()
    print(obj2.function2()[0], '\n')
    print('obj2

This solution is also close to idea, but has different characteristics than the reflection solution. The instrumentation implementation may have inaccuracies: for example, it may think that certain statements should be timed even when they should not be. Our particular implementation performs dumb analysis to determine when to instrument a statement, and can be improved to get a better accuracy, at the cost of making the implementation complex (and more bug prone). There are theoretical limits, as interesting code analysis in general is undeciadable to perform with perfect accuracy.

On the other hand, because we are instrumenting client code, our print statement are placed within client code. Given them access to a variety of useful context information. For example, the name of the instance of the library object being used. Notice the print statements above show that name. In general, instrumentation has access to a wealth of context information it can use to run sophisticated static analysis, and do interesting things.

Finally, note that because we instrument only client code. Internal calls within the library are not instrumented, and therefore not timed. Note how calls to function3 no longer produce timmings for internal calls to function1 and function2.

Below we will undo the instrumentation of the client code so it does not interfer with the last solution.

In [12]:
original_code = copy